## Segmentation & Tracking using DeepCell
- Ran originally on Google Colab

## Installation

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.fin('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/vanvalenlab/deepcell-tf.git

Cloning into 'deepcell-tf'...
remote: Enumerating objects: 5587, done.
remote: Counting objects: 100% (1142/1142), done.
remote: Compressing objects: 100% (549/549), done.
remote: Total 5587 (delta 765), reused 901 (delta 593), pack-reused 4445
Receiving objects: 100% (5587/5587), 335.98 MiB | 25.03 MiB/s, done.
Resolving deltas: 100% (3905/3905), done.


In [ ]:
cd deepcell-tf/

/content/deepcell-tf/deepcell-tf


In [1]:
# !pip install -e . 

In [ ]:
!pip install -Uqq imagecodecs
!pip install -Uqq "scikit-image == 0.19.3"

In [ ]:
!pip show deepcell-tracking

Name: DeepCell-Tracking
Version: 0.6.1
Summary: Tracking cells and lineage with deep learning.
Home-page: https://github.com/vanvalenlab/deepcell-tracking
Author: Van Valen Lab
Author-email: vanvalenlab@gmail.com
License: LICENSE
Location: /usr/local/lib/python3.7/dist-packages
Requires: pandas, scipy, deepcell-toolbox, scikit-image, numpy, networkx
Required-by: 


#### Our modified files to extract features from tracks
- Abvilable at the folder appended - deepcell_tracking

In [ ]:
!cp -rf /content/drive/MyDrive/DeepCell/utils.py /usr/local/lib/python3.7/dist-packages/deepcell_tracking

In [ ]:
!cp -rf /content/drive/MyDrive/DeepCell/tracking.py /usr/local/lib/python3.7/dist-packages/deepcell_tracking

### Choose Folder with TIF Images - Ours was on drive

In [ ]:
tif_path = Path(r'/content/drive/MyDrive/DeepCell/Images')
tif_files = list(tif_path.glob('*.tif'))

In [ ]:
len(tif_files)

10278

## Functions & Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio
import tifffile as tiff 
import pickle
from pathlib import Path
from skimage import io
from tensorflow.keras import backend as K

from datetime import datetime
from deepcell.applications import NuclearSegmentation
from deepcell.applications import CellTracking

In [ ]:
def get_cord_for_crop(tif_files):
    tff = tiff.imread(tif_files[0])
    Y_size = tff.shape[0]
    X_size = tff.shape[1]
    cord_list = []
    cord_list.append([0,int(X_size/2),0,int(Y_size/2)])
    cord_list.append([0,int(X_size/2),int(Y_size/2), Y_size])
    cord_list.append([int(X_size/2),X_size,0,int(Y_size/2)])
    cord_list.append([int(X_size/2),X_size,int(Y_size/2), Y_size])
    return cord_list

def merge_tff_even_crop(tif_files ,cord_list, seq_length = 0):
    if cord_list:
        x1,x2,y1,y2 = cord_list
    if tif_files:
        tff = tiff.imread(tif_files[0])
        tff = tff[y1:y2,x1:x2]
        if len(tff.shape) == 2:
            tff = np.expand_dims(tff, axis=-1)
        tff = np.expand_dims(tff, axis=0)
    if tif_files[1:]:
        for i, other in enumerate(tif_files[1:]):
            if i%2 == 0:
                continue
            othertff = tiff.imread(other)
            othertff = othertff[y1:y2,x1:x2]
            if len(othertff.shape) == 2:
                othertff = np.expand_dims(othertff, axis=-1)
            othertff = np.expand_dims(othertff, axis=0)
            tff = np.concatenate((tff,othertff))

    if seq_length > 0 and seq_length < len(tff):
        tff = tff[0:seq_length,...]
    return tff
    
def merge_tff(tif_files ,even=True, seq_length = 0):
    if tif_files:
        tff = tiff.imread(tif_files[0])
        if len(tff.shape) == 2:
            tff = np.expand_dims(tff, axis=-1)
        tff = np.expand_dims(tff, axis=0)
    if tif_files[1:]:
        for i, other in enumerate(tif_files[1:]):
            if even and i%2 == 0:
                continue
            othertff = tiff.imread(other)
            if len(othertff.shape) == 2:
                othertff = np.expand_dims(othertff, axis=-1)
            othertff = np.expand_dims(othertff, axis=0)
            tff = np.concatenate((tff,othertff))

    if seq_length > 0 and seq_length < len(tff):
        tff = tff[0:seq_length,...]
    return tff

def pick_channel(x,channel = 0):
    return x[...,channel:(channel+1)]


def segmentation(x, file_path, mpp =1.24):
    app = NuclearSegmentation()
    y_seg = app.predict(x, image_mpp = mpp)
    np.save(file_path, y_seg, allow_pickle=True)
    return y_seg

def tracking(x, y_seg):
    tracker = CellTracking()
    tracked_data = tracker.track(np.copy(x), y_seg)
    # d = tracked_data.copy()
    # np.save(file_name, d, allow_pickle=True)
    return tracked_data

def plot_tff(im1,im2,vmin,vmax):
    fig, ax = plt.subplots(1, 2, figsize=(30, 15))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Tracked')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image


def save_results(x,y_seg, tracked_data, tff_path,well,txt):
    centroids = pd.DataFrame(columns = range(x.shape[0]))
    morphologies = pd.DataFrame(columns = range(x.shape[0]))
    embeddings = pd.DataFrame(columns = range(x.shape[0]))

    for cell_id, cell_dict in tracked_data['tracks'].items():
        for i,frame in enumerate(cell_dict['frames']):
            centroids.at[cell_id,frame] = cell_dict['centroid'][i]
            morphologies.at[cell_id,frame] = cell_dict['morphologies'][i]
            embeddings.at[cell_id,frame] = cell_dict['embedding'][i]

    date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    directory = f"Results_{well}_{txt}_{date}"
    res_path = tff_path.joinpath(directory)
    os.mkdir(res_path)

    centroids.to_csv(res_path.joinpath('centroids.csv'))
    morphologies.to_csv(res_path.joinpath('morphologies.csv'))
    embeddings.to_csv(res_path.joinpath('embeddings.csv'))

    # with open(res_path.joinpath('track_results.pkl'), 'wb') as f:
    #     pickle.dump(tracked_data, f)

    X = tracked_data['X']  # raw X data
    y = tracked_data['y_tracked']  # tracked y data

    # imageio.mimsave(res_path.joinpath('tracks.tif'), [plot_tff(x[i,...,0], y[i,...,0], y.min(), y.max())
    #                             for i in range(y_seg.shape[0])])
    imageio.mimsave(res_path.joinpath('tracks.gif'), [plot_tff(x[i,...,0], y[i,...,0], y.min(), y.max())
                        for i in range(y_seg.shape[0])])

## Main

### Merging The TIF files to 1 TIF file
- The files will be sorted by wells, and then views
- the tiff file will have 4 dimensions :
  - (t,x,y,c) 
    - t = time 
    - x - x location
    - y - y location
    - c - number of channels

In [ ]:
tif_files = list(tif_path.glob('*.tif'))
well_dict = {}
for tif_file in tif_files:
    file_name = tif_file.stem.split('_')
    well_name = file_name[2]
    view_name = file_name[3]
    
    if well_name not in well_dict:
        well_dict[well_name] = {}
    if view_name not in well_dict[well_name]:
        well_dict[well_name][view_name] = []

    well_dict[well_name][view_name].append(tif_file)

for well, views in well_dict.items():
    for view , view_tif in views.items():
        print(f'well:{well} , view:{view}, {len(view_tif)}')

well:D2 , view:3, 95
well:D2 , view:2, 95
well:D2 , view:4, 95
well:D2 , view:1, 95
well:D3 , view:1, 95
well:D3 , view:2, 95
well:D3 , view:3, 95
well:D3 , view:4, 95
well:G5 , view:1, 95
well:G5 , view:4, 95
well:G5 , view:2, 84
well:G5 , view:3, 95
well:G10 , view:4, 16
well:G2 , view:2, 95
well:G2 , view:4, 95
well:G2 , view:3, 95
well:G2 , view:1, 95
well:E7 , view:1, 95
well:E7 , view:2, 95
well:E7 , view:4, 95
well:E7 , view:3, 95
well:F2 , view:2, 95
well:F2 , view:3, 95
well:F2 , view:4, 95
well:F2 , view:1, 95
well:E4 , view:2, 95
well:E4 , view:1, 95
well:G6 , view:1, 95
well:G6 , view:2, 85
well:G6 , view:3, 25
well:F9 , view:1, 190
well:F9 , view:4, 190
well:F9 , view:2, 189
well:F9 , view:3, 190
well:E10 , view:1, 95
well:E10 , view:2, 95
well:E10 , view:3, 95
well:E10 , view:4, 95
well:E9 , view:1, 95
well:E9 , view:4, 95
well:E9 , view:2, 95
well:E9 , view:3, 95
well:F8 , view:1, 190
well:F8 , view:4, 190
well:F8 , view:2, 190
well:F8 , view:3, 190
well:E8 , view:1, 95


### Segmenting all wells - utilizing GPU for speedup

In [ ]:
for well, views in well_dict.items():
    for view , view_tif in views.items():
        file_path = f'/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_{well}_{view}.npy'
        my_file = Path(file_path)
        if my_file.is_file():
            continue
        if len(view_tif) > 95:
            continue
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        merged_tff = merge_tff(view_tif,even=False)
        segmentation(merged_tff,file_path)
        del merged_tff

### Tracking all segmented wells - dividing the TIFFs to 4 quarters (crops)
- to avoid MemoryExauhstion Error

In [ ]:
res_path = Path(f'/content/drive/MyDrive/DeepCell/Results')
for well, views in well_dict.items():
    for view , view_tif in views.items():
        if well == 'D2':
          continue
        if list(res_path.glob(f'Results_{well}_{view}_*')):
          continue
        e = 0
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        file_path = f'/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_{well}_{view}.npy'
        res_path_folder = f'/content/drive/MyDrive/DeepCell/Results'
        my_file = Path(file_path)
        if my_file.is_file():
          print(file_path)
          seg_tff = np.load(file_path,allow_pickle=True)
        else:  
          continue
          # seg_tff = segmentation(merged_tff,file_path)
        if seg_tff.shape[0] < 95:
            print(f"Seg shape:{seg_tff.shape[0]} and merge {len(view_tif)} mismatch")
            continue
        # Take files from folder
        merged_tff = merge_tff(view_tif,even=False)
        # Even
        merged_tff = merged_tff[::2,...]
        seg_tff = seg_tff[::2,...]
        if seg_tff.shape[0] != merged_tff.shape[0]:
            print("Seg and merge mismatch")
            continue
        # Crop
        cord_list = get_cord_for_crop(view_tif)
        for i,cord in enumerate(cord_list):
            print(cord)
            x1,x2,y1,y2 = cord
            merged_tff_c = merged_tff[:,y1:y2,x1:x2,...]
            seg_tff_c = seg_tff[:,y1:y2,x1:x2,...]
            # Check not empty frames
            l = np.array([np.max(seg_tff_c[i,...]) for i in range(seg_tff_c.shape[0])])
            if len(np.argwhere(l==0)) > 0:
                e = np.max(np.argwhere(l==0)) + 1
            else:
                e = 0
            if e == seg_tff_c.shape[0]:
                print(f"No cells in the crop {i}")
                continue
            print(e)
            # From Frames with cells
            merged_tff_c = merged_tff_c[e:,...]
            seg_tff_c = seg_tff_c[e:,...]
            print("Tracking")
            track_tff_c = tracking(merged_tff_c,seg_tff_c)
            save_results(merged_tff_c,seg_tff_c,track_tff_c,res_path,well,f'{view}_start_{e}_crop_{i}')
            del track_tff_c
            del seg_tff_c


well:G5 , view:1, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_G5_1.npy
Seg shape:23 and merge 95 mismatch
well:G5 , view:2, 84
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_G5_2.npy
Seg shape:84 and merge 84 mismatch
well:G10 , view:4, 16
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_G10_4.npy
Seg shape:16 and merge 16 mismatch
well:G2 , view:2, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_G2_2.npy
Seg shape:2 and merge 95 mismatch
well:G2 , view:4, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_G2_4.npy
Seg shape:4 and merge 95 mismatch
well:E7 , view:1, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_E7_1.npy
Seg shape:2 and merge 95 mismatch
well:E7 , view:2, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_E7_2.npy
Seg shape:28 and merge 95 mismatch
well:E7 , view:4, 95
/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_E7_4.npy
Seg shape:13 and merge 95 mismatch
well:F2 , view:2, 95
/content/drive/MyDrive/DeepCell/Segm

In [ ]:
# for well, views in well_dict.items():
#     for view , view_tif in views.items():
#         # if well == 'D2' and view =='2':
#         #   continue
#         e = 0
#         print(f'well:{well} , view:{view}, {len(view_tif)}')
#         view_tif.sort()
#         file_path = f'/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_{well}_{view}.npy'
#         res_path_folder = f'/content/drive/MyDrive/DeepCell/Results'
#         my_file = Path(file_path)
#         if my_file.is_file():
#           print(file_path)
#           seg_tff = np.load(file_path,allow_pickle=True)
#         else:  
#           continue
#           # seg_tff = segmentation(merged_tff,file_path)
#         if seg_tff.shape[0] < 95:
#             continue
#         # Take files from folder
#         merged_tff = merge_tff(view_tif,even=False)
#         # Even
#         merged_tff = merged_tff[::2,...]
#         seg_tff = seg_tff[::2,...]
#         if seg_tff.shape[0] != merged_tff.shape[0]:
#             continue
#             # Check not empty frames
#         l = np.array([np.max(seg_tff[i,...]) for i in range(seg_tff.shape[0])])
#         if len(np.argwhere(l==0)) > 0:
#             e = np.max(np.argwhere(l==0)) + 1
#         else:
#             e = 0
#         if e == seg_tff.shape[0]:
#             continue
#         print(e)
#         # From Frames with cells
#         merged_tff = merged_tff[e:,...]
#         seg_tff= seg_tff[e:,...]
#         print("Tracking")
#         track_tff = tracking(merged_tff,seg_tff)
#         save_results(merged_tff,seg_tff,track_tff,res_path,well,f'{view}_start_{e}')
#         del track_tff
#         del seg_tff


In [ ]:
file_path = f'/content/drive/MyDrive/DeepCell/Segmentation/seg_tff_D2_2.npy'
seg_tff = np.load(file_path,allow_pickle=True)
seg_tff.shape

In [ ]:
file_path = f'/content/drive/MyDrive/DeepCell/seg_tff_{well}_{view}.npy'
for well, views in well_dict.items():
    for view , view_tif in views.items():
        if well != 'D2' or view !='2':
          continue
        print(f'well:{well} , view:{view}, {len(view_tif)}')
        view_tif.sort()
        file_path = f'/content/drive/MyDrive/DeepCell/seg_tff_{well}_{view}.npy'

        cord_list = get_cord_for_crop(view_tif)
        for i,cord in enumerate(cord_list):
            # print(cord)
            # merged_tff = merge_tff_even_crop(view_tif,cord) # , seq_length = 30
            merged_tff = merge_tff(view_tif)
            # merged_tff = merged_tff[5:,...]
            segmentation(merged_tff,1.24)
            # print(merged_tff.shape)
            # print("Segmentation")
            # p = Process(target=segmentation ,args=(merged_tff,1.24,q) )
            # p.start()
            # p.join()
            # seg_tff = np.load(file_name,allow_pickle=True)
            # print(seg_tff.shape)
            # #(Drop if is empty)
            # print("Tracking")
            # p = multiprocessing.Process(target= tracking ,args=(merged_tff,seg_tff,) )
            # p.start()
            # p.join()
            # track_tff =  np.load(file_name,allow_pickle=True).item()
            # print(track_tff.keys())
            # save_results(merged_tff,seg_tff,track_tff,tif_path,well,f'{view}_crop_{i}')
